In [5]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px

In [6]:
engine = sqlite3.connect("Group_Project_3.db")

### Review the tables


In [7]:
query= "SELECT name FROM sqlite_master WHERE type='table'"
table= pd.DataFrame(engine.execute(query),columns=["project3_table"])
table

,project3_table
0,stock
1,Apple_price
2,Apple_volume
3,Google_price
4,Google_volume
5,Microsoft_price
6,Microsoft_volume
7,Intel_price
8,Intel_volume
9,Oracle_price


### Review the Apple_price table


In [8]:
table_name="Apple_price"
query=f"Select * from {table_name}"
value=engine.execute(query)
query=f"""SELECT name FROM PRAGMA_TABLE_INFO('{table_name}')"""
column=engine.execute(query)
clean_column=[]
for item in list(column):
    clean_column.append(item[0])
temp=pd.DataFrame(value,columns=clean_column)
temp["date"]=pd.to_datetime(temp["date"])
temp.set_index("date",inplace=True)
temp

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,AAPL,130.28,130.90,124.17,125.07,1,1.0000,NaN,NaN
2023-01-04,AAPL,126.89,128.66,125.08,126.36,1,1.0103,0.0103,1.0103
2023-01-05,AAPL,127.13,127.77,124.76,125.02,1,0.9996,-0.0106,0.9996
2023-01-06,AAPL,126.01,130.29,124.89,129.62,1,1.0364,0.0368,1.0364
2023-01-09,AAPL,130.47,133.41,129.89,130.15,1,1.0406,0.0041,1.0406
...,...,...,...,...,...,...,...,...,...
2023-12-22,AAPL,195.18,195.41,192.97,193.60,12,1.5479,-0.0055,1.5479
2023-12-26,AAPL,193.61,193.89,192.83,193.05,12,1.5435,-0.0028,1.5435
2023-12-27,AAPL,192.49,193.50,191.09,193.15,12,1.5443,0.0005,1.5443


### Create a function to work for each table

In [10]:
def Database_to_Frame(table_name):
    query=f"Select * from {table_name}"
    value=engine.execute(query)
    query=f"""SELECT name FROM PRAGMA_TABLE_INFO('{table_name}')"""
    column=engine.execute(query)
    clean_column=[]
    for item in list(column):
        clean_column.append(item[0])
    temp=pd.DataFrame(value,columns=clean_column)
    temp["date"]=pd.to_datetime(temp["date"])
    temp.set_index("date",inplace=True)
    return temp

In [11]:
Google=Database_to_Frame("Google_price")
Google

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,GOOGL,89.59,91.05,88.52,89.12,1,1.0000,NaN,NaN
2023-01-04,GOOGL,90.35,90.65,87.27,88.08,1,0.9883,-0.0117,0.9883
2023-01-05,GOOGL,87.47,87.57,85.90,86.20,1,0.9672,-0.0213,0.9672
2023-01-06,GOOGL,86.79,87.69,84.86,87.34,1,0.9800,0.0132,0.9800
2023-01-09,GOOGL,88.36,90.05,87.86,88.02,1,0.9877,0.0078,0.9877
...,...,...,...,...,...,...,...,...,...
2023-12-22,GOOGL,140.77,141.99,140.71,141.49,12,1.5876,0.0076,1.5876
2023-12-26,GOOGL,141.59,142.68,141.19,141.52,12,1.5880,0.0002,1.5880
2023-12-27,GOOGL,141.59,142.08,139.89,140.37,12,1.5751,-0.0081,1.5751


In [12]:
Google_volume=Database_to_Frame("Google_volume")
Google_volume

,stock_id,Adj Close,Volume
date,,,
2023-01-03,GOOGL,89.120003,28131200
2023-01-04,GOOGL,88.080002,34854800
2023-01-05,GOOGL,86.199997,27194400
2023-01-06,GOOGL,87.339996,41381500
2023-01-09,GOOGL,88.019997,29003900
...,...,...,...
2023-12-22,GOOGL,141.490005,26514600
2023-12-26,GOOGL,141.520004,16780300
2023-12-27,GOOGL,140.369995,19628600


In [24]:
def bubble(df):
    temp=df.dropna()
    fig=px.scatter(temp,x=temp.index, y="Percentage_change",size=temp["Percentage_change"].abs(),color=temp["Percentage_change"])
    return fig


In [25]:
bubble(Google).show()
# For example in Google:the bigger the circle or the color, the bigger the % change
# the negative number means we are losing money and in that situation the color gets money.
# On Oct. 25th, the investor lost 8% of the money

In [26]:
Apple=Database_to_Frame("Apple_price")
Apple

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,AAPL,130.28,130.90,124.17,125.07,1,1.0000,NaN,NaN
2023-01-04,AAPL,126.89,128.66,125.08,126.36,1,1.0103,0.0103,1.0103
2023-01-05,AAPL,127.13,127.77,124.76,125.02,1,0.9996,-0.0106,0.9996
2023-01-06,AAPL,126.01,130.29,124.89,129.62,1,1.0364,0.0368,1.0364
2023-01-09,AAPL,130.47,133.41,129.89,130.15,1,1.0406,0.0041,1.0406
...,...,...,...,...,...,...,...,...,...
2023-12-22,AAPL,195.18,195.41,192.97,193.60,12,1.5479,-0.0055,1.5479
2023-12-26,AAPL,193.61,193.89,192.83,193.05,12,1.5435,-0.0028,1.5435
2023-12-27,AAPL,192.49,193.50,191.09,193.15,12,1.5443,0.0005,1.5443


In [27]:
bubble(Apple).show()

In [28]:
Intel=Database_to_Frame("Intel_price")
Intel

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,INTC,27.05,27.07,26.37,26.73,1,1.0000,NaN,NaN
2023-01-04,INTC,27.41,27.92,27.16,27.68,1,1.0355,0.0355,1.0355
2023-01-05,INTC,27.44,27.78,27.38,27.56,1,1.0311,-0.0043,1.0311
2023-01-06,INTC,27.93,28.83,27.38,28.73,1,1.0748,0.0425,1.0748
2023-01-09,INTC,28.84,29.87,28.84,29.31,1,1.0965,0.0202,1.0965
...,...,...,...,...,...,...,...,...,...
2023-12-22,INTC,47.25,48.16,47.20,48.00,12,1.7957,0.0195,1.7957
2023-12-26,INTC,48.92,50.52,48.71,50.50,12,1.8893,0.0521,1.8893
2023-12-27,INTC,50.63,51.28,50.19,50.76,12,1.8990,0.0051,1.8990


In [29]:
bubble(Intel).show()

In [30]:
Oracle=Database_to_Frame("Oracle_price")
Oracle

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,ORCL,82.47,83.84,82.26,83.72,1,1.0000,NaN,NaN
2023-01-04,ORCL,84.20,85.17,83.64,84.48,1,1.0091,0.0091,1.0091
2023-01-05,ORCL,85.00,85.40,83.22,84.31,1,1.0070,-0.0020,1.0070
2023-01-06,ORCL,84.74,86.34,83.82,85.66,1,1.0232,0.0160,1.0232
2023-01-09,ORCL,86.84,87.59,86.16,86.42,1,1.0323,0.0089,1.0323
...,...,...,...,...,...,...,...,...,...
2023-12-22,ORCL,105.70,106.49,105.46,106.20,12,1.2685,0.0032,1.2685
2023-12-26,ORCL,106.52,106.65,105.34,106.19,12,1.2684,-0.0001,1.2684
2023-12-27,ORCL,106.32,106.33,105.56,105.94,12,1.2654,-0.0024,1.2654


In [31]:
bubble(Oracle).show()

In [33]:
Microsoft=Database_to_Frame("Microsoft_price")
Microsoft

,stock_id,Open,High,Low,Close,Month,normalization,Percentage_change,accumulation
date,,,,,,,,,
2023-01-03,MSFT,243.08,245.75,237.40,239.58,1,1.0000,NaN,NaN
2023-01-04,MSFT,232.28,232.87,225.96,229.10,1,0.9563,-0.0437,0.9563
2023-01-05,MSFT,227.20,227.55,221.76,222.31,1,0.9279,-0.0296,0.9279
2023-01-06,MSFT,223.00,225.76,219.35,224.93,1,0.9389,0.0118,0.9389
2023-01-09,MSFT,226.45,231.24,226.41,227.12,1,0.9480,0.0097,0.9480
...,...,...,...,...,...,...,...,...,...
2023-12-22,MSFT,373.68,375.18,372.71,374.58,12,1.5635,0.0028,1.5635
2023-12-26,MSFT,375.00,376.94,373.50,374.66,12,1.5638,0.0002,1.5638
2023-12-27,MSFT,373.69,375.06,372.81,374.07,12,1.5614,-0.0016,1.5614


In [35]:
bubble(Microsoft).show()

In [39]:
def candleplot(candle,title="candle_plot_for_stock"):
    fig=go.Figure(data=[go.Candlestick(x=candle.index,open=candle["Open"],close=candle["Close"], high=candle["High"], low=candle["Low"], 
                        increasing_line_color="green", decreasing_line_color="red")])
    fig.update_layout(title=title)
    return fig

In [41]:
candleplot(Google,"Google_stock")

In [42]:
candleplot(Apple,"Apple_stock")

In [43]:
candleplot(Oracle,"Oracle_stock")

In [44]:
candleplot(Intel,"Intel_stock")

In [45]:
candleplot(Microsoft,"Microsoft_stock")